<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-03-24 11:01:12
-------------------
qualified stocks: 90
with latest results: 84
still star stocks: 46
-------------------
Initial Investment:  1.06 C
CY Investment:  1.19 C
Reserve:  4.00 K
Current:  1.13 C
-------------------
Today PnL: 1.42 L (1.28%)
Current PnL: -20.28 L (-17.02%)
CY Booked + Current PnL: -6.41 L (-5.38%)
-------------------
Total profit:  3.49 L
Total loss:  -23.77 L
-------------------
Total Booked + Current PnL: 6.70 L (6.32%)
Total Booked PnL: 26.98 L (25.44%)
Curr Year Booked PnL: 13.87 L (12.29%)
Prev Year Booked PnL: 13.11 L (12.36%)
Est FTT:  1.88 C
Est FTT PnL: 75.63 L (67.03%)
-------------------
Est LTT:  2.49 C
Est LTT PnL: 1.36 C (120.4%)
Deployed:  1.06 C
Current:  1.13 C
CAGR/XIRR %: 4.57%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
55      NMDC    65.62    82.00    40.27       M-MC  75.56  104790.0   
12  AXISBANK   986.10  1317.30   -19.87       H-LC  84.44  112460.0   
69   SBILIFE  1483.67  1924.99   -10.82       H-LC  68.89  107243.0   
70      SBIN   760.30   863.00   -21.41       M-LC  76.67  203838.0   
47      LICI   750.57   983.00   -21.42       H-LC  47.78  106865.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
55       4391.0  20675.0        1.45          4.37  19.73  24.96  507.0   
12      11878.0  21907.0        3.03         11.81  19.48  33.59    NaN   
69       6353.0  23658.0        1.99          6.30  22.06  29.75    NaN   
70       2358.0  24848.0        2.12          1.17  12.19  13.51   93.0   
47       6289.0  24857.0        0.78          6.25  23.26  30.97  202.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
55     0.21        0.93    18.11     XY25      NTT     MINING  
12     0.54        1.00    16.29      X40      ATH      BANKS  
69     0.27        0.95    18.02       AR      ATH  INSURANCE  
70     0.09        1.81    11.67     XY25      NTT      BANKS  
47     0.25        0.95     9.37     XY25      NTT  INSURANCE

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    FTT  Dev%_PE Conviction     RSP   Current  \
0       5PAISA   564.75  810.0   -49.35       H-SC   63.33  128411.0   
58  RAJESHEXPO   517.65  583.0   810.21       M-SC   97.78   53437.0   
88  VALIANTORG   512.64  838.0  -239.36       H-SC   64.44   75543.0   
46     LAOPALA   369.40  464.0   -24.67       H-SC   57.78   63606.0   
22   COFFEEDAY    59.14   80.0  -126.76       L-SC  100.00   56832.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
0      -58521.0  139698.0       10.12        -31.31  108.79  43.43  20.0   
58     -83740.0  101055.0        7.71        -61.05  189.11  12.62 -49.0   
88     -57743.0  142338.0        6.92        -43.32  188.42  63.47 -27.0   
46     -37240.0   63065.0        5.08        -36.93   99.15  25.61 -50.0   
22     -56717.0   96768.0        4.48        -49.95  170.27  35.27   NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
0     -0.42        1.14    22.85     X40N      BTT    FINANCE  
58    -0.83        0.47    32.90     X40N      NTT  JEWELLERY  
88    -0.41        0.67    26.63       XR      NTT  CHEMICALS  
46    -0.59        0.56    14.72       AR      NTT   CERAMICS  
22    -0.59        0.50    39.10       XR      NTT     HOTELS

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
29    GICRE   335.06   583.0     8.65       H-MC  93.33  191909.0   
43   JIOFIN   320.24   387.0   -29.73       M-LC  36.67  113185.0   
51  MEDANTA  1022.60  1486.0     9.34       X-MC  54.44  190798.0   
40     INFY  1436.24  2275.0   -14.93       X-LC   5.56  164434.0   
24    DABUR   518.36   735.0   -10.97       X-LC  23.33  128342.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
29      45488.0  62869.0       -1.39         31.07  32.76  74.00  109.0   
43     -45014.0  77996.0       -1.11        -28.45  68.91  20.85   26.0   
51      35363.0  35069.0       -0.78         22.75  18.38  45.32    8.0   
40      15065.0  72170.0       -0.72         10.09  43.89  58.40   33.0   
24      -3840.0  59089.0       -0.69         -2.90  46.04  41.79    1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
29     0.72        1.70    40.35     XY24      BTT   INSURANCE  
43    -0.58        1.00    14.04     XY24      BTT     FINANCE  
51     1.01        1.69    29.82     XY24      NTT  HEALTHCARE  
40     0.21        1.46    14.74      X40      BTT          IT  
24    -0.06        1.14     4.05     XY24      BTT        FMCG

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
54    NHPC    82.08   115.00   208.53       M-MC  83.33  144140.0   
55    NMDC    65.62    82.00    40.27       M-MC  75.56  104790.0   
29   GICRE   335.06   583.00     8.65       H-MC  93.33  191909.0   
49  LUXIND  1448.19  2442.00     3.89       M-SC  71.11  138358.0   
20    CAMS  3643.00  5250.99    -3.31       H-SC  91.11  107449.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
54       3127.0  53433.0        1.62          2.22  37.07  40.11  -54.0   
55       4391.0  20675.0        1.45          4.37  19.73  24.96  507.0   
29      45488.0  62869.0       -1.39         31.07  32.76  74.00  109.0   
49        780.0  93627.0        1.47          0.57  67.67  68.62    6.0   
20       5445.0  39584.0        0.57          5.34  36.84  44.14  261.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
54     0.06        1.28    17.84     XY24      NTT      POWER  
55     0.21        0.93    18.11     XY25      NTT     MINING  
29     0.72        1.70    40.35     XY24      BTT  INSURANCE  
49     0.01        1.23    35.68     X40N      NTT   TEXTILES  
20     0.14        0.95    33.58       XR      ATH       MISC

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
20        CAMS  3643.00  5250.99    -3.31       H-SC  91.11  107449.0   
14   BANKINDIA   112.90   190.00   -31.68       M-MC  73.33   95718.0   
75    SONACOMS   578.05  1006.00   -32.15       M-MC  32.22   88891.0   
1     AARTIIND   487.04   919.00    10.11       M-SC  31.11   68245.0   
65  SAMMAANCAP   170.35   326.00  -160.68       M-SC  51.11   72204.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
20       5445.0   39584.0        0.57          5.34   36.84  44.14  261.0   
14      -4537.0   73004.0        1.84         -4.53   76.27  68.29  -12.0   
75     -12268.0   87158.0       -0.28        -12.13   98.05  74.03   -8.0   
1      -13091.0   85231.0        0.34        -16.09  124.89  88.69   -8.0   
65     -30006.0  123397.0        2.03        -29.36  170.90  91.37    NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
20     0.14        0.95    33.58       XR      ATH       MISC  
14    -0.06        0.85    19.30       XR      NTT      BANKS  
75    -0.14        0.79     7.12       AR      BTT       AUTO  
1     -0.15        0.61     8.68       XR      NTT  CHEMICALS  
65    -0.24        0.64    14.49       XR      NTT    FINANCE

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
51   MEDANTA  1022.60  1486.0     9.34       X-MC  54.44  190798.0   
29     GICRE   335.06   583.0     8.65       H-MC  93.33  191909.0   
67   SBICARD   765.91  1075.0    -8.90       H-MC  43.89  200514.0   
2      AAVAS  1815.63  2328.0   -28.91       M-SC  98.89  213781.0   
34  HDFCBANK  1549.96  2170.0    -2.28       X-LC  77.78  378084.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
51      35363.0  35069.0       -0.78         22.75  18.38  45.32    8.0   
29      45488.0  62869.0       -1.39         31.07  32.76  74.00  109.0   
67      20525.0  52114.0       -0.50         11.40  25.99  40.36   -1.0   
2       19509.0  35317.0        0.85         10.04  16.52  28.22    6.0   
34      52592.0  77621.0        1.70         16.16  20.53  40.00    4.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
51     1.01        1.69    29.82     XY24      NTT  HEALTHCARE  
29     0.72        1.70    40.35     XY24      BTT   INSURANCE  
67     0.39        1.78    28.92     XY24      NTT     FINANCE  
2      0.55        1.90    51.86     XY24      NTT     FINANCE  
34     0.68        3.35    28.03      X40      BTT       BANKS

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
22   COFFEEDAY    59.14    80.0  -126.76       L-SC  100.00   56832.0   
2        AAVAS  1815.63  2328.0   -28.91       M-SC   98.89  213781.0   
58  RAJESHEXPO   517.65   583.0   810.21       M-SC   97.78   53437.0   
27  EASEMYTRIP    18.68    26.4     0.18       M-SC   96.67  122040.0   
57       QUESS   711.66   986.0   -24.84       M-SC   95.56  152665.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
22     -56717.0   96768.0        4.48        -49.95  170.27  35.27   NaN   
2       19509.0   35317.0        0.85         10.04   16.52  28.22   6.0   
58     -83740.0  101055.0        7.71        -61.05  189.11  12.62 -49.0   
27     -44361.0  113131.0        1.18        -26.66   92.70  41.33 -91.0   
57      -3189.0   63264.0        2.06         -2.05   41.44  38.55 -33.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
22    -0.59        0.50    39.10       XR      NTT     HOTELS  
2      0.55        1.90    51.86     XY24      NTT    FINANCE  
58    -0.83        0.47    32.90     X40N      NTT  JEWELLERY  
27    -0.39        1.08    24.09     XY24      NTT     TRAVEL  
57    -0.05        1.35    41.15     XY24      NTT       MISC

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       20.02
1     25       43.22
2     50       72.15

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    41.93
X40     14.31
X40N    11.71
AR      11.43
XR       9.15
XY25     9.02
X200     1.21
SR       1.20
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    21.54
M-SC    20.13
H-SC    18.46
M-MC     7.90
M-LC     6.81
X-MC     6.36
H-MC     6.36
H-LC     5.38
X-SC     2.76
L-SC     2.50
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               14.67        -11.87   63.22
FINANCE             10.90        -18.02   66.53
FMCG                10.19         -5.87   37.96
IT                   7.43        -32.83   99.03
MISC                 7.34        -21.93   64.27
INSURANCE            6.27          3.14   42.41
PAINTS               5.86        -24.34   61.13
CHEMICALS            3.94        -21.78  102.04
ELECTRICAL           3.41        -10.83   57.13
AUTO                 3.09        -18.39   70.45
HEALTHCARE           2.94         -1.28   36.16
FOOTWEAR             2.66        -37.09   82.39
POWER                2.30         -9.07   52.70
TRAVEL               2.09        -30.38   83.62
TYRES                2.02          0.27   31.88
REFINERIES           1.82          0.72   17.57
TEXTILES             1.80        -13.54   73.17
STEEL                1.79        -11.19   94.06
CERAMICS             1.57        -59.97  125.04
JEWELLERY            1.39        -84.50  142.85
CEMENT               1.25        -33.83  102.69
ENGINEERING          1.18        -37.11   89.94
ENTERTAINMENT        0.98        -47.63  121.87
MINING               0.93          4.19   19.73
AC                   0.91        -24.30   92.92
DURABLES             0.73        -23.19   84.04
HOTELS               0.50        -99.80  170.27

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3143625.0     31
XR        1203297.0     14
X40N      1144003.0     15
AR         956826.0     11
X40        634547.0     10
SR         209184.0      2
XY25       189762.0      6
X200        81326.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2175442.0     18
M-SC        2005794.0     22
X-LC         845178.0     14
M-MC         653144.0      7
X-SC         366218.0      4
H-MC         337854.0      5
L-SC         314905.0      4
X-MC         299947.0      5
M-LC         270930.0      5
H-LC         188695.0      5

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
M-SC       XY24      948539.0      9
H-SC       XY24      920696.0      6
           X40N      539808.0      6
M-MC       XY24      472307.0      4
M-SC       XR        452121.0      5
X-LC       X40       417467.0      6
H-SC       AR        389228.0      2
M-SC       X40N      365141.0      5
H-SC       XR        325710.0      4
H-MC       XY24      274147.0      4
M-LC       XY24      246082.0      4
X-LC       AR        167914.0      3
L-SC       XR        165930.0      2
X-SC       SR        127164.0      1
           XR        122825.0      1
X-MC       X40N      122825.0      2
X-LC       XY25      119382.0      3
X-SC       X40N      116229.0      2
M-SC       AR         88134.0      1
M-MC       AR         87158.0      1
H-LC       AR         85885.0      2
L-SC       XY24       83233.0      1
M-SC       SR         82020.0      1
X-LC       X200       81326.0      1
H-LC       X40        77953.0      2
M-MC       XR         73004.0      1
X-MC       AR         72765.0      1
M-SC       X40        69839.0      1
X-MC       X40        69288.0      1
L-SC       AR         65742.0      1
H-MC       XR         63707.0      1
X-LC       XY24       59089.0      1
X-MC       XY24       35069.0      1
H-LC       XY25       24857.0      1
M-LC       XY25       24848.0      1
M-MC       XY25       20675.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
